# Imports

In [1]:
import numpy as np
import pandas as pd
import os

/var/folders/z_/7fqm4zqs11zdv7cm41c6w_pc0000gn/T/ipykernel_16461/3339900087.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Read usable data
Read the usables.csv file, and filter out items that are not downloaded, and thus unavailable.

In [2]:
df = pd.read_csv("/Users/balazsmorvay/PycharmProjects/VAE/Data/usables.csv", sep=';')
for i in range(len(df)):
    filename = df.loc[i, 'FILE_ID']
    filepath = os.path.join('/Users/balazsmorvay/Downloads/ABIDE/data/Outputs/ccs/filt_noglobal/func_preproc',
                            f'{filename}_func_preproc.nii.gz')
    df.loc[i, "EXISTS"] = 1 if os.path.exists(filepath) else 0
df = df[df.EXISTS == 1]
df.head()

,FILE_ID,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,HANDEDNESS_CATEGORY,SITE_ID,EXISTS
1,Pitt_0050004,1,1,1,19.09,R,PITT,1
2,Pitt_0050005,1,1,2,13.73,R,PITT,1
3,Pitt_0050006,1,1,1,13.37,L,PITT,1
4,Pitt_0050007,1,1,1,17.78,R,PITT,1
5,Pitt_0050008,1,1,1,32.45,R,PITT,1


# Check number of data points in sites

In [3]:
df.drop(df[df.FILE_ID == "UM_1_0050302"].index, inplace=True)

In [4]:
df['SITE_ID'].value_counts()

SITE_ID
NYU         169
UM_1         80
USM          60
UCLA_1       55
YALE         47
TRINITY      44
MAX_MUN      42
PITT         41
KKI          39
STANFORD     33
SDSU         31
UM_2         31
LEUVEN_1     28
LEUVEN_2     26
OLIN         25
SBL          23
OHSU         22
UCLA_2       20
CALTECH      15
CMU           2
Name: count, dtype: int64

# Select the highest, and create train, test and val.csv files

In [52]:
df = df[df.SITE_ID == 'NYU']
timesteps = np.arange(0, 175)
df = df.merge(pd.Series(timesteps, name='TIME_SLICE'), how='cross')
train_df = df.sample(frac=0.8,random_state=42)
remaining = df.drop(train_df.index)
val_df = remaining.sample(frac=0.5, random_state=42)
test_df = remaining.drop(val_df.index)

In [53]:
train_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,TIME_SLICE
count,23660.000000,23660.000000,23660.000000,23660.000000,23660.0,23660.000000
mean,1.571682,0.583178,1.208749,15.389336,1.0,86.964455
std,0.494845,0.774212,0.406423,6.613007,0.0,50.485709
min,1.000000,0.000000,1.000000,6.470000,1.0,0.000000
25%,1.000000,0.000000,1.000000,10.520000,1.0,43.000000
50%,2.000000,0.000000,1.000000,13.710000,1.0,87.000000
75%,2.000000,1.000000,1.000000,19.640000,1.0,131.000000
max,2.000000,3.000000,2.000000,39.100000,1.0,174.000000


In [54]:
val_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,TIME_SLICE
count,2958.000000,2958.000000,2958.000000,2958.000000,2958.0,2958.000000
mean,1.578431,0.573360,1.202840,15.447469,1.0,86.748479
std,0.493894,0.774461,0.402182,6.546772,0.0,50.400657
min,1.000000,0.000000,1.000000,6.470000,1.0,0.000000
25%,1.000000,0.000000,1.000000,10.650000,1.0,42.000000
50%,2.000000,0.000000,1.000000,13.720000,1.0,88.000000
75%,2.000000,1.000000,1.000000,19.730000,1.0,130.000000
max,2.000000,3.000000,2.000000,39.100000,1.0,174.000000


In [55]:
test_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,TIME_SLICE
count,2957.000000,2957.000000,2957.000000,2957.000000,2957.0,2957.000000
mean,1.587758,0.560027,1.198174,15.375878,1.0,87.536016
std,0.492322,0.769117,0.398691,6.560503,0.0,50.907407
min,1.000000,0.000000,1.000000,6.470000,1.0,0.000000
25%,1.000000,0.000000,1.000000,10.520000,1.0,43.000000
50%,2.000000,0.000000,1.000000,13.710000,1.0,89.000000
75%,2.000000,1.000000,1.000000,19.640000,1.0,132.000000
max,2.000000,3.000000,2.000000,39.100000,1.0,174.000000


In [56]:
train_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE/train.csv')
val_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE/val.csv')
test_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE/test.csv')

# Select the second highest to be the out of distribution test set

In [5]:
um_df = df[df.SITE_ID == 'UM_1']
um_df.head()

,FILE_ID,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,HANDEDNESS_CATEGORY,SITE_ID,EXISTS
179,UM_1_0050272,1,1,1,14.2,R,UM_1,1
180,UM_1_0050273,1,1,1,16.8,R,UM_1,1
181,UM_1_0050274,1,1,1,14.2,R,UM_1,1
182,UM_1_0050275,1,1,1,11.5,L,UM_1,1
183,UM_1_0050276,1,1,2,16.8,-9999,UM_1,1


In [9]:
timesteps = np.arange(0, 295)
um_df = um_df.merge(pd.Series(timesteps, name='TIME_SLICE'), how='cross')

In [10]:
um_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,TIME_SLICE
count,23600.000000,23600.000000,23600.000000,23600.000000,23600.0,23600.00000
mean,1.575000,-124.487500,1.275000,13.913750,1.0,147.00000
std,0.494353,1110.993193,0.446524,2.866319,0.0,85.16048
min,1.000000,-9999.000000,1.000000,8.200000,1.0,0.00000
25%,1.000000,0.000000,1.000000,11.500000,1.0,73.00000
50%,2.000000,0.000000,1.000000,13.850000,1.0,147.00000
75%,2.000000,1.000000,2.000000,16.100000,1.0,221.00000
max,2.000000,3.000000,2.000000,19.200000,1.0,294.00000


In [11]:
um_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE/UM_1.csv')